In [100]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from backtesting import Backtest, Strategy
from strategies import PairTradeMeanReversion
import numpy as np
import pandas as pd
import numpy as np
import yfinance as yf
np.int = int

In [150]:
def create_pair(df_1: str, df_2: str) -> pd.DataFrame:
    df_1.index = pd.DatetimeIndex(df_1.index)
    df_2.index = pd.DatetimeIndex(df_2.index)
    pair_df = df_1.join(df_2, rsuffix='_y').dropna()
    pair_df.index = pair_df.index.tz_localize(None).astype('datetime64[ns]')
    return pair_df.reset_index()


In [180]:
import common.util as util

params = None

class PairTradeMeanReversion(Strategy):

    def init(self):
        self.outliers = self.I(util.flag_outliers, self.data.df, 'Ratio', 2)

    def next(self):
        if (self.outliers[-1] < 0 and not params['is_denom']) or (self.outliers[-1] > 0 and params['is_denom']):
            self.buy(size=0.1)
        if (self.outliers[-1] > 0 and not params['is_denom']) or (self.outliers[-1] < 0 and params['is_denom']):
            self.sell(size=0.1)
        if abs(round(self.data['Ratio'][-1], 2)-round(self.data.df['Ratio'].mean(), 2)) == 0.1:
            self.position.close()

pairs = [
    ['AAPL', 'MSFT', ['2020-01-01', None]],
    ['META', 'GOOG', ['2020-01-01', None]],
    ['F', 'GM', ['2006-01-01', '2010-01-01']],
    ['BA', 'LMT', ['2012-01-01', None]],
    ['MRK', 'LLY', ['2007-01-01', '2018-01-01']],
]

for pair in pairs:
    print(pair)
    tkr_1 = pair[0]
    tkr_2 = pair[1]
    e_start = 50000

    df_1 = yf.Ticker(tkr_1).history(start=pair[2][0], end=pair[2][1])
    df_2 = yf.Ticker(tkr_2).history(start=pair[2][0], end=pair[2][1])

    params = {'is_denom': False}
    pair_1 = create_pair(df_1, df_2)
    ratio = util.norm_pairs(pair_1, 'Close', 'Close_y')
    pair_1['Ratio'] = ratio.copy()
    pair_1_bt = Backtest(pair_1, PairTradeMeanReversion,
                        cash=e_start, commission=.002,
                        exclusive_orders=True)
    output_1 = pair_1_bt.run()

    params = {'is_denom': True}
    pair_2 = create_pair(df_2, df_1)
    pair_2['Ratio'] = ratio.copy()
    pair_2_bt = Backtest(pair_2, PairTradeMeanReversion,
                        cash=e_start, commission=.002,
                        exclusive_orders=True)
    output_2 = pair_2_bt.run()

    print(f"is valid: {output_1['# Trades']==output_2['# Trades']}")
    e_1 = output_1['Equity Final [$]']
    e_2 = output_2['Equity Final [$]']
    print(f"Final pair equity: {(e_1+e_2)/(2*e_start)*100}")
    print(f"Pair return %: {(e_1+e_2)/(2*e_start)*100}")
    print(f"wr 1 %: {output_1['Win Rate [%]']}, wr 2 %:{output_2['Win Rate [%]']}")
    bh_1 = df_1['Close'][-1]-df_1['Close'][0]
    bh_2 = df_2['Close'][-1]-df_2['Close'][0]
    print(f"Buy and hold return %: {(bh_1+bh_2)/(df_1['Close'][0]+df_2['Close'][0])*100}\n")

['AAPL', 'MSFT', ['2020-01-01', None]]
is valid: True
Final pair equity: 97.47260080873549
Pair return %: 97.47260080873549
wr 1 %: 42.10526315789473, wr 2 %:50.877192982456144
Buy and hold return %: 157.68477718409076

['META', 'GOOG', ['2020-01-01', None]]


GM: Data doesn't exist for startDate = 1136091600, endDate = 1262322000


is valid: True
Final pair equity: 91.79126128324891
Pair return %: 91.79126128324891
wr 1 %: 40.50632911392405, wr 2 %:49.36708860759494
Buy and hold return %: 121.13367682023004

['F', 'GM', ['2006-01-01', '2010-01-01']]


TypeError: Cannot join tz-naive with tz-aware DatetimeIndex